In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
    .master("local[*]")\
    .appName('Pyspark_ex')\
    .getOrCreate()

In [6]:
df= spark.read.csv("name_gender.csv")

In [7]:
!wget https://s3-geospatial.s3-us-west-2.amazonaws.com/1800.csv


--2023-07-04 22:51:01--  https://s3-geospatial.s3-us-west-2.amazonaws.com/1800.csv
Resolving s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)... 52.218.243.65, 3.5.82.174, 52.218.193.57, ...
Connecting to s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)|52.218.243.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62728 (61K) [text/csv]
Saving to: ‘1800.csv’

1800.csv            100%[===================>]  61.26K   189KB/s    in 0.3s    

2023-07-04 22:51:02 (189 KB/s) - ‘1800.csv’ saved [62728/62728]



In [10]:
!head -5 1800.csv

ITE00100554,18000101,TMAX,-75,,,E,
ITE00100554,18000101,TMIN,-148,,,E,
GM000010962,18000101,PRCP,0,,,E,
EZE00100082,18000101,TMAX,-86,,,E,
EZE00100082,18000101,TMIN,-135,,,E,


In [19]:
import pandas as pd

pd_df = pd.read_csv(
    "1800.csv",
    names=['stationID','data','measure_type','temperature'],
    usecols=[0,1,2,3]
)
pd_dfb

,stationID,data,measure_type,temperature
0,ITE00100554,18000101,TMAX,-75
1,ITE00100554,18000101,TMIN,-148
2,GM000010962,18000101,PRCP,0
3,EZE00100082,18000101,TMAX,-86
4,EZE00100082,18000101,TMIN,-135
...,...,...,...,...
1820,ITE00100554,18001231,TMAX,50
1821,ITE00100554,18001231,TMIN,25
1822,GM000010962,18001231,PRCP,16
1823,EZE00100082,18001231,TMAX,14


In [22]:
pd_mintemp = pd_df[pd_df["measure_type"]=="TMIN"]

In [23]:
pd_stationtemp = pd_mintemp[["stationID","temperature"]]

In [30]:
pd_stationtemp.groupby(["stationID"]).max()

,temperature
stationID,
EZE00100082,204
ITE00100554,241


In [12]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 55.9 MB/s eta 0:00:00 0:00:01
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
  Using cached numpy-1.25.0-cp39-cp39-macosx_11_0_arm64.whl (14.0 MB)


In [31]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

conf = SparkConf()
conf.set("spark.app.name","soomers DataFrame")
conf.set("spark.master","local[*]")

spark = SparkSession.builder\
    .config(conf=conf)\
    .getOrCreate()

23/07/04 23:15:41 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [32]:
df = spark.read.format('csv').load('1800.csv') # spark.read.csv('1800.csv')

In [34]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)



In [35]:
df = spark.read.format('csv')\
    .load('1800.csv')\
    .toDF("stationID","date",'measure_tpye','temperature','_c4','_c5','_c6','_c7')

In [36]:
df.printSchema()

root
 |-- stationID: string (nullable = true)
 |-- date: string (nullable = true)
 |-- measure_tpye: string (nullable = true)
 |-- temperature: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)



In [38]:
df = spark.read.format('csv')\
    .option('inferSchema','true')\
    .load('1800.csv')\
    .toDF("stationID","date",'measure_tpye','temperature','_c4','_c5','_c6','_c7')

In [40]:
df.printSchema()

root
 |-- stationID: string (nullable = true)
 |-- date: integer (nullable = true)
 |-- measure_tpye: string (nullable = true)
 |-- temperature: integer (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)



In [41]:
from pyspark.sql.types import  StringType, IntegerType, FloatType
from pyspark.sql.types import StructField, StructType

# struct type 안에 field 반복
# struct Field는 필드명, 타입, null값 허용 여부
schema = StructType([\
    StructField('stationID',StringType(),True),\
    StructField('date',IntegerType(),True),\b
    StructField('measure_type',StringType(),True),\
    StructField('temperature',FloatType(),True)])

In [42]:
df = spark.read.schema(schema).csv('1800.csv')

In [43]:
df.printSchema()

root
 |-- stationID: string (nullable = true)
 |-- date: integer (nullable = true)
 |-- measure_type: string (nullable = true)
 |-- temperature: float (nullable = true)



In [45]:
minTemp = df.filter(df.measure_type == "TMIN")

In [48]:
minTemp2 = df.where("measure_type = 'TMIN'") # SQL expression

In [52]:
minTemp3 = df.where(df.measure_type == 'TMIN')


In [53]:
minTemp3.show()

+-----------+--------+------------+-----------+
|  stationID|    date|measure_type|temperature|
+-----------+--------+------------+-----------+
|ITE00100554|18000101|        TMIN|     -148.0|
|EZE00100082|18000101|        TMIN|     -135.0|
|ITE00100554|18000102|        TMIN|     -125.0|
|EZE00100082|18000102|        TMIN|     -130.0|
|ITE00100554|18000103|        TMIN|      -46.0|
|EZE00100082|18000103|        TMIN|      -73.0|
|ITE00100554|18000104|        TMIN|      -13.0|
|EZE00100082|18000104|        TMIN|      -74.0|
|ITE00100554|18000105|        TMIN|       -6.0|
|EZE00100082|18000105|        TMIN|      -58.0|
|ITE00100554|18000106|        TMIN|       13.0|
|EZE00100082|18000106|        TMIN|      -57.0|
|ITE00100554|18000107|        TMIN|       10.0|
|EZE00100082|18000107|        TMIN|      -50.0|
|ITE00100554|18000108|        TMIN|       14.0|
|EZE00100082|18000108|        TMIN|      -31.0|
|ITE00100554|18000109|        TMIN|       23.0|
|EZE00100082|18000109|        TMIN|     

In [50]:
minTemp2.show()

+-----------+--------+------------+-----------+
|  stationID|    date|measure_type|temperature|
+-----------+--------+------------+-----------+
|ITE00100554|18000101|        TMIN|     -148.0|
|EZE00100082|18000101|        TMIN|     -135.0|
|ITE00100554|18000102|        TMIN|     -125.0|
|EZE00100082|18000102|        TMIN|     -130.0|
|ITE00100554|18000103|        TMIN|      -46.0|
|EZE00100082|18000103|        TMIN|      -73.0|
|ITE00100554|18000104|        TMIN|      -13.0|
|EZE00100082|18000104|        TMIN|      -74.0|
|ITE00100554|18000105|        TMIN|       -6.0|
|EZE00100082|18000105|        TMIN|      -58.0|
|ITE00100554|18000106|        TMIN|       13.0|
|EZE00100082|18000106|        TMIN|      -57.0|
|ITE00100554|18000107|        TMIN|       10.0|
|EZE00100082|18000107|        TMIN|      -50.0|
|ITE00100554|18000108|        TMIN|       14.0|
|EZE00100082|18000108|        TMIN|      -31.0|
|ITE00100554|18000109|        TMIN|       23.0|
|EZE00100082|18000109|        TMIN|     

In [54]:
min_temp_by_station = minTemp.groupby("stationID").min("temperature")

In [58]:
station_temp = minTemp[["stationID","temperature"]]

In [59]:
station_temp.show()

+-----------+-----------+
|  stationID|temperature|
+-----------+-----------+
|ITE00100554|     -148.0|
|EZE00100082|     -135.0|
|ITE00100554|     -125.0|
|EZE00100082|     -130.0|
|ITE00100554|      -46.0|
|EZE00100082|      -73.0|
|ITE00100554|      -13.0|
|EZE00100082|      -74.0|
|ITE00100554|       -6.0|
|EZE00100082|      -58.0|
|ITE00100554|       13.0|
|EZE00100082|      -57.0|
|ITE00100554|       10.0|
|EZE00100082|      -50.0|
|ITE00100554|       14.0|
|EZE00100082|      -31.0|
|ITE00100554|       23.0|
|EZE00100082|      -46.0|
|ITE00100554|       31.0|
|EZE00100082|      -75.0|
+-----------+-----------+
only showing top 20 rows



In [60]:
results = min_temp_by_station.collect()

In [65]:
for result in results:
    print(result[0],result[1])

ITE00100554 -148.0
EZE00100082 -135.0


In [66]:
df.createOrReplaceTempView("station1800")


In [75]:
sql_results = spark.sql("""
SELECT stationID, MIN(temperature) min
FROM station1800
WHERE measure_type = 'TMIN'
GROUP BY 1""")
sql_results.show()

+-----------+------+
|  stationID|   min|
+-----------+------+
|ITE00100554|-148.0|
|EZE00100082|-135.0|
+-----------+------+



In [69]:
my_result = sql_results.collect()

In [71]:
for i in my_result:

    print(i[0],i[1])

ITE00100554 -148.0
EZE00100082 -135.0
